## 一、LLaMA-Factory

### 1、介绍
<img src="LLaMA-Factory.png" style="margin-left: 0px" width="600px">

LLaMA Factory 是一个可视化的大语言模型微调平台，可以在无需编写任何代码的前提下，在本地完成上百种预训练模型的微调。


项目地址：[hiyouga/LLaMA-Factory: Unified Efficient Fine-Tuning of 100+ LLMs (ACL 2024) (github.com)](https://github.com/hiyouga/LLaMA-Factory)


相关论文：[2403.13372 (arxiv.org)](https://arxiv.org/pdf/2403.13372)


### 2、本地安装

```bash
git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
cd LLaMA-Factory
pip install -e ".[torch,metrics]"
```

### 3、运行webui

```bash

# 前台运行
export USE_MODELSCOPE_HUB=1 && llamafactory-cli webui

# 挂后台运行
export USE_MODELSCOPE_HUB=1 && nohup llamafactory-cli webui > /dev/null 2>&1 &
```

其中，`USE_MODELSCOPE_HUB`设为1，下载模型时会使用ModelScope的镜像加速，避免从HuggingFace下载模型可能存在的网络问题。



打开链接：[http://10.104.60.47:7860/](http://10.104.60.47:7860/) ，webui界面如下


<img src="webui.png" style="margin-left: 0px" width="1000px">

## 二、使用LLaMA-Factory微调qwen2.5模型

### 1、数据集格式介绍

LLaMA-Factory目前支持 alpaca 和 sharegpt 格式的数据集。

#### alpaca格式

```json
[
  {
    "instruction": "人类指令（必填）",
    "input": "人类输入（选填）",
    "output": "模型回答（必填）",
    "system": "系统提示词（选填）",
    "history": [
      ["第一轮指令（选填）", "第一轮回答（选填）"],
      ["第二轮指令（选填）", "第二轮回答（选填）"]
    ]
  }
]

```

在指令监督微调时，`instruction` 内容会与 `input` 内容拼接后作为人类指令，即人类指令为 `instruction\ninput`。而 `output` 内容为模型回答。

`system` 内容将被作为系统提示词。

`history` 由多个字符串二元组构成的列表，分别代表历史消息中每轮对话的指令和回答。注意在指令监督微调时，历史消息中的回答内容也会被用于模型学习。

#### sharegpt格式

```json
[
  {
    "conversations": [
      {
        "from": "human",
        "value": "人类指令"
      },
      {
        "from": "function_call",
        "value": "工具参数"
      },
      {
        "from": "observation",
        "value": "工具结果"
      },
      {
        "from": "gpt",
        "value": "模型回答"
      }
    ],
    "system": "系统提示词（选填）",
    "tools": "工具描述（选填）"
  }
]
```

相比 `alpaca` 格式的数据集，`sharegpt` 格式支持更多的角色种类，例如 `human、gpt、observation、function` 等等。

注意其中 `human` 和 `observation` 必须出现在奇数位置，`gpt` 和 `function` 必须出现在偶数位置。


### 2、自定义数据集准备（使用alpaca格式）

先将原始的csv格式数据转换为alpaca格式数据，代码详见：`alpaca_data_generator.py`

In [ ]:
import json
import pandas as pd
import os


def process_csv_files(input_paths, output_file, eval_ratio=0.1,
                      instruction='你是一个医疗方面的专家，可以根据患者的问题进行解答。'):
    # 创建一个空列表，用于存储转换后的数据
    alpaca_data = []

    # 遍历文件列表，读取并处理每一个CSV文件
    for csv_file in input_paths:
        # 读取当前CSV文件
        df = pd.read_csv(csv_file, encoding='ANSI')
        # 随机打乱数据
        df = df.sample(frac=1).reset_index(drop=True)

        # 遍历每一行数据，并将其转换为AlpaCA格式
        for index, row in df.iterrows():
            data = {
                "instruction": instruction,
                "input": row['ask'],
                "output": row['answer']
            }
            alpaca_data.append(data)

    # 将数据划分出训练集和验证集
    train_data = alpaca_data[:int(len(alpaca_data) * (1 - eval_ratio))]
    eval_data = alpaca_data[int(len(alpaca_data) * (1 - eval_ratio)):]

    print(f"数据已处理完毕，共 {len(alpaca_data)} 条数据，其中 {len(train_data)} 条用于训练，{len(eval_data)} 条用于验证")

    # 将训练集和验证集数据保存为JSON格式文件，文件分开保存
    with open(f"data/{output_file}_train.json", "w", encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=4)
        print(f"训练集数据已保存到 {f.name}")
    with open(f"data/{output_file}_eval.json", "w", encoding='utf-8') as f:
        json.dump(eval_data, f, ensure_ascii=False, indent=4)
        print(f"验证集数据已保存到 {f.name}")


# 配置文件路径
data_paths = [
    os.path.join("E:/Chinese-medical-dialogue-data", "样例_内科5000-6000.csv"),
    # os.path.join("E:/Chinese-medical-dialogue-data/Data_数据/Pediatric_儿科", "儿科5-14000.csv"),
    # os.path.join("E:/Chinese-medical-dialogue-data/Data_数据/Surgical_外科", "外科5-14000.csv"),
    # os.path.join("E:/Chinese-medical-dialogue-data/Data_数据/IM_内科", "内科5000-33000.csv"),
]

output_file_name = "chinese-medical-dialogue"

# 处理并保存为JSON文件
process_csv_files(data_paths, output_file_name)

转换后的数据格式如下：

<img src="chinese-medical-dialogue.png" style="margin-left: 0px" width="1000px">

将转换好的数据放到LLaMA-Factory的`data`目录下，并在该目录下的`dataset_info.json` 文件中提供您的数据集定义。



对于alpaca格式的数据，dataset_info.json 中的数据集描述应为：

```json
"数据集名称": {
  "file_name": "data.json",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output",
    "system": "system",
    "history": "history"
  }
}
```



对于shaegpt格式的数据，dataset_info.json 中的数据集描述应为：

```json
"数据集名称": {
  "file_name": "data.json",
  "formatting": "sharegpt",
  "columns": {
    "messages": "conversations",
    "system": "system",
    "tools": "tools"
  }
}
```



本实验使用alpaca格式，数据集描述示例如下：

```json
"chinese-medical-dialogue_train": {
  "file_name": "chinese-medical-dialogue_train.json",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}
```

上述步骤处理完成后，应该就可以在webui的数据集列表中找到我们自定义的数据集

<img src="data_list.png" style="margin-left: 0px" width="1000px">


### 3、实验设置



1、选择微调的原模型

2、选择模型路径：支持填写本地模型路径，如`/root/fangxy/qwen2.5-fine-tuning/model/qwen/Qwen2___5-1___5B-Instruct`；也支持填写huggingface的模型标识符，如qwen/Qwen2.5-1.5B-Instruct（自动下载模型并保存在`/root/.cache/modelscope/hub/...`目录下）

3、选择微调方法，默认lora

4、（可选）如果有之前训练过的检查点（checkpoints），可以在这里选择，没有就空着。

<img src="training_set1.png" style="margin-left: 0px" width="1000px">

5、高级设置，默认保持不变

<img src="training_set2.png" style="margin-left: 0px" width="1000px">


6、选择数据集，其他训练参数按需调整

<img src="training_set3.png" style="margin-left: 0px" width="1000px">

7、点击开始按钮进行微调训练
<img src="training_set4.png" style="margin-left: 0px" width="1000px">


### 4、实验过程展示

<img src="training_set5.png" style="margin-left: 0px" width="1000px">


### 5、模型评估

选择【Evaluate & Predict】选项卡开始模型评估

1、选择验证数据集

2、按需调整验证过程参数

3、开始

<img src="eval_webui1.png" style="margin-left: 0px" width="1000px">


评估结果：ROUGE分数衡量了模型输出答案（predict）和验证集中的标准答案（label）的相似度，ROUGE分数越高代表模型学习得越好。

<img src="eval_result.png" style="margin-left: 0px" width="1000px">



### 6、开始聊天测试

选择【Chat】选项卡开始聊天

1、选择模型名称和模型路径

2、选择检查点路径：指定一个微调后的模型路径

3、推理引擎默认使用huggingface，推理数据类型auto

4、加载模型

5、在聊天框中开始聊天
<img src="chat_webui.png" style="margin-left: 0px" width="1000px">


### 7、模型导出

在【Export】选项卡中操作：

1、指定一个导出目录

2、开始导出

<img src="model_export1.png" style="margin-left: 0px" width="1000px">


导出的结果：

<img src="model_export2.png" style="margin-left: 0px" width="1000px">


## 其他补充内容



### 1、GPU显存要求

以Qwen2.5-7B-Instruct模型为例，模型文件大小约为15.23G，全量微调所需显存约为4倍模型文件大小，LoRA微调显存约为模型文件大小

下图是LLaMA-Factory给出的不同微调方式和模型精度所需的GPU显存

<img src="gpu_req.png" style="margin-left: 0px" width="1000px">


### 2、模型量化

1、模型量化（Quantization）是指将模型的权重从高精度（如32位浮点数）转换为较低精度（如16位浮点数或8位整数）**以减少模型大小、降低内存占用和加速推理计算**的技术。

2、LLaMA-Factory微调参数中有切换模型精度的参数

<img src="计算类型.png" style="margin-left: 0px" width="400px">



3、模型精度分类

（1）FP32 (32-bit Floating Point)：32 位单精度浮点数，由1位符号位、8位指数位、23位尾数位组成；

（2）FP16 (16-bit Floating Point)：16 位半精度浮点数，由1位符号位、5位指数位、10位尾数位组成；

（3）BF16 (16-bit Brain Floating Point)：16 位浮点数，由1位符号位、8位指数位、7位尾数位组成；

（4）FP8 (8-bit Floating Point)：8 位浮点数，1 位符号位，剩余的7位可以在指数位和尾数位之间以不同方式分配；

（5）INT8、INT4


4、不同显卡对模型精度的支持情况不同

> https://blog.csdn.net/u011119817/article/details/120055088


<img src="precision.png" style="margin-left: 0px" width="1000px">


